# Predicting personality from patterns of behavior collected with smartphones

## Importing the libraries

In [ ]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.model_selection import RepeatedKFold
from numpy import arange
import math
import scipy
from sklearn.metrics import r2_score
import csv

## Preprocessing

In [ ]:
mdf = pd.read_csv("final_dataset.csv", decimal=",") #The final_dataset is the Pre-processed dataset

In [ ]:
mdf.head()

,Unnamed: 0,userId,entropy_apps_freq,entropy_apps_dur,huberM_dur_apps,Qn_dur_apps,anonymized_app_feature_2,daily_mean_num_unique_apps,daily_mean_num_unique_apps_week,daily_mean_num_unique_apps_weekend,...,C3.Sense.of.duty,C4.Ambition,C5.Discipline,C6.Caution,A1.Willingness.to.trust,A2.Genuineness,A3.Helpfulness,A4.Obligingness,A5.Modesty,A6.Good.naturedness
0,0,01ADD53B,2.207808471252917,2.18287051070534,4.2077409085690265,3.2758363899907987,0.0,14.2,15.045454545454545,11.875,...,1.495,1.579,2.447,-0.457,1.875,1.990,0.200,-0.822,2.202,-0.538
1,1,01C997E4,2.0376132825654216,1.941876060016405,4.633551122043685,3.942472096974795,0.0,9.733333333333333,10.318181818181818,8.125,...,0.455,0.630,0.563,0.016,4.211,3.585,-0.540,3.705,0.093,2.094
2,2,03E6965E,1.9307098415222836,2.142898501425948,7.145426927960067,5.655167792314818,0.0,18.703703703703702,19.473684210526315,16.875,...,0.954,0.630,0.862,0.834,1.123,0.862,1.047,1.606,0.516,1.223
3,3,0436543F,2.5205230790384467,2.5108222602045123,5.7834472014743925,4.932419535451934,0.0,9.066666666666666,11.4375,11.125,...,1.782,1.579,1.816,1.139,-0.257,1.121,1.351,1.322,1.923,1.223
4,4,0674C0C3,1.74767398013136,1.3703864271325918,8.676489153975304,7.066710025722562,0.0,0.5333333333333333,0.5909090909090909,0.375,...,0.221,-0.193,-1.571,1.768,-0.705,-0.061,0.753,-0.822,-0.892,0.389


In [ ]:
mdf.shape

(630, 1890)

In [ ]:
mdf.columns

Index(['Unnamed: 0', 'userId', 'entropy_apps_freq', 'entropy_apps_dur',
       'huberM_dur_apps', 'Qn_dur_apps', 'anonymized_app_feature_2',
       'daily_mean_num_unique_apps', 'daily_mean_num_unique_apps_week',
       'daily_mean_num_unique_apps_weekend',
       ...
       'C3.Sense.of.duty', 'C4.Ambition', 'C5.Discipline', 'C6.Caution',
       'A1.Willingness.to.trust', 'A2.Genuineness', 'A3.Helpfulness',
       'A4.Obligingness', 'A5.Modesty', 'A6.Good.naturedness'],
      dtype='object', length=1890)

In [ ]:
mdf = mdf.dropna()

In [ ]:
label_encoder = preprocessing.LabelEncoder()
mdf['gender']= label_encoder.fit_transform(mdf['gender'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
datatypes = mdf.dtypes

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth', None) 

In [ ]:
datatypes

Unnamed: 0                                                         int64
userId                                                            object
entropy_apps_freq                                                 object
entropy_apps_dur                                                  object
huberM_dur_apps                                                   object
Qn_dur_apps                                                       object
anonymized_app_feature_2                                          object
daily_mean_num_unique_apps                                        object
daily_mean_num_unique_apps_week                                   object
daily_mean_num_unique_apps_weekend                                object
app_simi_dayNight                                                 object
app_simi_weekWeekend                                              object
IV_apps                                                           object
daily_mean_num_foto                                

In [ ]:
mdf = mdf.drop(['userId'], axis = 1)

In [ ]:
v1 = lambda x:float(x[1:-1])

In [ ]:
mdf.head()

,Unnamed: 0,entropy_apps_freq,entropy_apps_dur,huberM_dur_apps,Qn_dur_apps,anonymized_app_feature_2,daily_mean_num_unique_apps,daily_mean_num_unique_apps_week,daily_mean_num_unique_apps_weekend,app_simi_dayNight,...,C3.Sense.of.duty,C4.Ambition,C5.Discipline,C6.Caution,A1.Willingness.to.trust,A2.Genuineness,A3.Helpfulness,A4.Obligingness,A5.Modesty,A6.Good.naturedness
0,0,2.207808471252917,2.18287051070534,4.2077409085690265,3.2758363899907987,0.0,14.2,15.045454545454545,11.875,0.9959345367404241,...,1.495,1.579,2.447,-0.457,1.875,1.990,0.200,-0.822,2.202,-0.538
1,1,2.0376132825654216,1.941876060016405,4.633551122043685,3.942472096974795,0.0,9.733333333333333,10.318181818181818,8.125,0.8876391325487125,...,0.455,0.630,0.563,0.016,4.211,3.585,-0.540,3.705,0.093,2.094
2,2,1.9307098415222836,2.142898501425948,7.145426927960067,5.655167792314818,0.0,18.703703703703702,19.473684210526315,16.875,0.9953016963912272,...,0.954,0.630,0.862,0.834,1.123,0.862,1.047,1.606,0.516,1.223
3,3,2.5205230790384467,2.5108222602045123,5.7834472014743925,4.932419535451934,0.0,9.066666666666666,11.4375,11.125,0.9417831950822276,...,1.782,1.579,1.816,1.139,-0.257,1.121,1.351,1.322,1.923,1.223
4,4,1.74767398013136,1.3703864271325918,8.676489153975304,7.066710025722562,0.0,0.5333333333333333,0.5909090909090909,0.375,0.8703028237186105,...,0.221,-0.193,-1.571,1.768,-0.705,-0.061,0.753,-0.822,-0.892,0.389


In [ ]:
mdf = mdf.astype(float)

In [ ]:
columns = list(mdf)


In [ ]:
columns

['Unnamed: 0',
 'entropy_apps_freq',
 'entropy_apps_dur',
 'huberM_dur_apps',
 'Qn_dur_apps',
 'anonymized_app_feature_2',
 'daily_mean_num_unique_apps',
 'daily_mean_num_unique_apps_week',
 'daily_mean_num_unique_apps_weekend',
 'app_simi_dayNight',
 'app_simi_weekWeekend',
 'IV_apps',
 'daily_mean_num_foto',
 'daily_mean_num_video',
 'daily_sd_num_foto',
 'daily_sd_num_video',
 'huberM_dur_Messaging',
 'huberM_dur_night_Messaging',
 'daily_mean_num_Academia',
 'huberM_dur_Private_Transportation',
 'huberM_dur_night_Private_Transportation',
 'IV_Tools',
 'Qn_dur_Tools',
 'daily_mean_num_unique_Tools',
 'daily_mean_num_unique_Tools_week',
 'anonymized_app_feature_7',
 'anonymized_app_feature_9',
 'daily_mean_num_Gallery',
 'daily_mean_num_unique_Gallery_weekend',
 'entropy_cat_dur_Gallery',
 'entropy_cat_freq_Gallery',
 'entropy_cat_dur_Sleep',
 'entropy_cat_freq_Sleep',
 'daily_mean_num_Sleep',
 'daily_mean_num_unique_Sleep_weekend',
 'huberM_dur_Gaming_Simulation',
 'huberM_dur_night

In [ ]:
mdf.dtypes

Unnamed: 0                                                       float64
entropy_apps_freq                                                float64
entropy_apps_dur                                                 float64
huberM_dur_apps                                                  float64
Qn_dur_apps                                                      float64
anonymized_app_feature_2                                         float64
daily_mean_num_unique_apps                                       float64
daily_mean_num_unique_apps_week                                  float64
daily_mean_num_unique_apps_weekend                               float64
app_simi_dayNight                                                float64
app_simi_weekWeekend                                             float64
IV_apps                                                          float64
daily_mean_num_foto                                              float64
daily_mean_num_video                               

In [ ]:
col = list(mdf)

X contains all the features

In [ ]:
X = mdf[['age']]
for i in col:
  if i == "O":
    break;
  else:
    X[i] = mdf[i]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [ ]:
X

,age,Unnamed: 0,entropy_apps_freq,entropy_apps_dur,huberM_dur_apps,Qn_dur_apps,anonymized_app_feature_2,daily_mean_num_unique_apps,daily_mean_num_unique_apps_week,daily_mean_num_unique_apps_weekend,...,sd_music_loudness_weekend,sd_music_speechiness_weekend,sd_music_acousticness_weekend,sd_music_instrumentalness_weekend,sd_music_liveness_weekend,sd_music_valence_weekend,sd_music_tempo_weekend,sd_music_popularity_weekend,gender,education
0,25.0,0.0,2.207808,2.182871,4.207741,3.275836,0.0,14.200000,15.045455,11.875000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0
1,18.0,1.0,2.037613,1.941876,4.633551,3.942472,0.0,9.733333,10.318182,8.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2,20.0,2.0,1.930710,2.142899,7.145427,5.655168,0.0,18.703704,19.473684,16.875000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,24.0,3.0,2.520523,2.510822,5.783447,4.932420,0.0,9.066667,11.437500,11.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0
4,20.0,4.0,1.747674,1.370386,8.676489,7.066710,0.0,0.533333,0.590909,0.375000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
5,22.0,5.0,3.031442,2.830932,6.968253,5.311451,0.0,19.333333,19.909091,17.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0
7,18.0,7.0,2.812993,2.762999,5.565330,4.304511,0.0,30.034483,31.142857,27.125000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0
8,19.0,8.0,1.950355,2.254156,6.886397,5.174430,0.0,19.700000,20.545455,17.375000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
9,25.0,9.0,1.399418,2.210551,4.565784,3.528863,0.0,4.133333,5.111111,5.333333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0
10,31.0,10.0,2.811781,2.517716,8.000250,6.123536,0.0,17.700000,18.000000,16.875000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


## Random Forest Model along with 5-fold Cross Validation

In [ ]:
# step-1: create a cross-validation scheme

folds = KFold(n_splits = 5, shuffle = True, random_state = 100)

# step-2: specify range of hyperparameters to tune
hyper_params = [{'n_features_to_select': list(range(1, 14))}]

estimator = RandomForestRegressor()
           
param_grid = { 
            "n_estimators"      : [10,20,30],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "bootstrap": [True, False],
            }
# 3.2 call GridSearchCV()
model_cv = GridSearchCV(estimator , 
                        param_grid, 
                        scoring= 'r2', 
                        cv = folds, 
                        verbose = 1,
                        return_train_score=True)      

Creating empty list to store the evaluated metrics

In [ ]:
rmselist = {}
r2list = {}
rlist = {}

Fitting the model for every Y(Personality Dimension)

In [ ]:
for column in mdf.columns[1854:]:
  Y = mdf[column]
  x_train, x_test,y_train,y_test = train_test_split(X,Y,test_size =0.4)
  model_cv.fit(x_train, y_train)
  y_pred = model_cv.predict(x_test)
  print(column)
  print(math.sqrt(mean_squared_error(y_test,y_pred)))
  print(r2_score(y_test,y_pred))
  a,_ = scipy.stats.pearsonr(y_test,y_pred)
  print(a)
  rmselist[column] = math.sqrt(mean_squared_error(y_test,y_pred))
  r2list[column] = r2_score(y_test,y_pred)
  rlist[column] = a


Fitting 5 folds for each of 54 candidates, totalling 270 fits
O
0.6793141924683685
0.047563508090716855
0.21881115558650555
Fitting 5 folds for each of 54 candidates, totalling 270 fits
C
0.7372306554292369
0.03143186018852717
0.197771721216679
Fitting 5 folds for each of 54 candidates, totalling 270 fits
E
0.7393200526053351
0.04079821130370931
0.26454648591875346
Fitting 5 folds for each of 54 candidates, totalling 270 fits
A
0.7742890242384233
-0.026682744369840083
0.07917087418043489
Fitting 5 folds for each of 54 candidates, totalling 270 fits
ES
0.7552793443471972
-0.009933756197686883
0.08185518154185238
Fitting 5 folds for each of 54 candidates, totalling 270 fits
ES1.Carefreeness
1.2466863100737031
0.025392496758118055
0.18185027047231653
Fitting 5 folds for each of 54 candidates, totalling 270 fits
ES2.Equanimity
1.0573880945346437
-0.0348760604819196
0.07393957843547468
Fitting 5 folds for each of 54 candidates, totalling 270 fits
ES3.Positive.mood
1.454481868211342
-0.04368

Writing the evaluated metrics to csv files

In [ ]:

w = csv.writer(open("outputr2.csv", "w"))
for key, val in r2list.items():
    w.writerow([key, val])

In [ ]:
w = csv.writer(open("outputrmse.csv", "w"))
for key, val in rmselist.items():
    w.writerow([key, val])

In [ ]:
w = csv.writer(open("outputr_rf.csv", "w"))
for key, val in r2list.items():
    w.writerow([key, val])

## Elastic Net Linear Regression Model along with 5-cross Validation

In [ ]:
rmselist1 = {}
rlist1 = {}


In [ ]:


model = ElasticNet()

cv = KFold(n_splits = 5, shuffle = True, random_state = 100)

grid = dict()
grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
grid['l1_ratio'] = arange(0, 1, 0.01)

model_cv1 = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

for column in mdf.columns[1882:]:
  Y = mdf[column]
  x_train, x_test,y_train,y_test = train_test_split(X,Y,test_size =0.4)
  model_cv1.fit(x_train, y_train)
  y_pred = model_cv1.predict(x_test)
  print(column)
  print(math.sqrt(mean_squared_error(y_test,y_pred)))
  a,_ = scipy.stats.pearsonr(y_test,y_pred)
  print(a)
  rmselist1[column] = math.sqrt(mean_squared_error(y_test,y_pred))
  
  rlist1[column] = a

C6.Caution
1.3993908669594486
-0.0042147574664674
A1.Willingness.to.trust
1.5575808866552485
-0.03066561226578037
A2.Genuineness
1.01741637019373
-0.021312277310667514
A3.Helpfulness
1.4346445240342591
0.018542465498840036
A4.Obligingness
1.4122994217262104
-0.03131749614236261
A5.Modesty
1.0661340024137869
0.07939480300080996
A6.Good.naturedness
2.014615696132241
-0.08590073729252728


In [ ]:
w = csv.writer(open("output_rmse_elastic.csv", "w"))
for key, val in rmselist1.items():
    w.writerow([key, val])

In [ ]:
w = csv.writer(open("output_r_elastic.csv", "w"))
for key, val in rlist1.items():
    w.writerow([key, val])